In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot
import seaborn
from tqdm import tqdm
from scipy.stats import ttest_ind
from collections import Counter


In [2]:
def get_duplicates(input_list):
    procd = [''.join(i.split()).lower() for i in input_list]
    counts = Counter(input_list)
    dups = [k for k,v in counts.items() if v > 1]
#     print(dups)
    return dups

In [3]:
# Read in raw data
enzyme_order_df = pd.read_excel("data/initial_enzymes_1.xlsx")
sp_df = pd.read_excel("data/sp_prot_translations.xls")

excel_raw = pd.ExcelFile("data/191203_mastersheet_BGsubtracted_raw.xls")


In [4]:
sheet_names = ['mastersheet',
    'amylase_1',
    'amylase_2',
    'amylase_1_10x',
    'amylase_2_10x',
    'protease_1',
    'protease_2',
    'xylanase_1',
    'xylanase_1_50x',
    'lipase_1',
    'lipase_2',
    'positive_1',
    'positive_2',
    'positive_amy_10x',
    'positive_xyl_50x',
    'negative']

In [5]:
first_few = ['Glyc stock plate', 'Well location', 'Planned Protein', 'Planned SP', 'Sequence Name', 'Correct Construct', 'Full Sequencing', 'SP start', 'SP end', 'SP coverage', 'SP actual DNA', 'SP expected DNA', 'SP expected name', 'Protein start', 'Protein end', 'Protein coverage', 'Protein actual DNA', 'Protein expected DNA', 'Protein expected name', 'Unnamed: 19', 'SP expected name.1']
next_few = ['Glyc stock plate', 'Well location', 'Planned Protein', 'Planned SP', 'Sequence Name', 'Correct Construct', 'Full Sequencing', 'SP start', 'SP end', 'SP coverage', 'SP actual DNA', 'SP expected DNA', 'SP expected name', 'Protein start', 'Protein end', 'Protein coverage', 'Protein actual DNA', 'Protein expected DNA', 'Protein expected name', 'SP expected name.1', 'Unnamed: 20', 'Assay1']

In [6]:
# Check for duplicate columns
# actually this is built in to pandas, SP.expected name is doubled
# Checking for correct column names

# Amylase sheets have an extra column
amylase_firsts = ['Glyc stock plate', 'Well location', 'Planned Protein', 'Planned SP', 'Sequence Name', 'Correct Construct', 'Full Sequencing', 'SP start', 'SP end', 'SP coverage', 'SP actual DNA', 'SP expected DNA', 'SP expected name', 'Protein start', 'Protein end', 'Protein coverage', 'Protein actual DNA', 'Protein expected DNA', 'Protein expected name', 'Unnamed: 19', 'SP expected name.1']
other_firsts = ['Glyc stock plate', 'Well location', 'Planned Protein', 'Planned SP', 'Sequence Name', 'Correct Construct', 'Full Sequencing', 'SP start', 'SP end', 'SP coverage', 'SP actual DNA', 'SP expected DNA', 'SP expected name', 'Protein start', 'Protein end', 'Protein coverage', 'Protein actual DNA', 'Protein expected DNA', 'Protein expected name', 'SP expected name.1', 'Unnamed: 20', 'Assay1']

def get_duplicates(input_list):
    procd = [''.join(i.split()).lower() for i in input_list]
    counts = Counter(input_list)
    dups = [k for k,v in counts.items() if v > 1]
    return dups

all_cols = []

# Iterate through excel sheets, grab columns and check for correct data
for i in range(1,len(sheet_names)): # skip master sheet
    _df = excel_raw.parse(excel_raw.sheet_names[i])
    a = list(_df.columns)
    dups = get_duplicates(a)
    all_cols = all_cols + a
#     print(sheet_names[i])
    if i <= 4:
        assert a[:len(amylase_firsts)] == amylase_firsts, 'incorrect first columns'
#         print(a[len(amylase_firsts):])
    else:
        assert a[:len(other_firsts)] == other_firsts, 'incorrect first columns'
#         print(a[len(other_firsts):])
#     if 'Assay5' in a:
#         print(sheet_names[i])
#         print(a[len(other_firsts):])
#     print('----')
    assert len(dups) == 0, "Error: Duplicate column names"

In [7]:
new_cols = {'Unnamed: 19': 'blank_col',
            'Unnamed: 21': 'id_concatd',
            'Unnamed: 23': 'assay_1_value',
            'Unnamed: 25': 'assay_2_value',
            'Unnamed: 27': 'assay_3_value',
            'Unnamed: 29': 'assay_4_value',
            'Assay4-BG': 'Assay4- BG',    
}

# Data after amylase 2 doesn't have blank column 19 
new_cols_2 = {
            'Unnamed: 20': 'id_concatd',
            'Unnamed: 22': 'assay_1_value',
            'Unnamed: 24': 'assay_2_value',
            'Unnamed: 26': 'assay_3_value',
            'Unnamed: 28': 'assay_4_value',
            'Assay4-BG': 'Assay4- BG',    
}

# Because the single blank line in amylase only offsets the columns by 1, 
# and there is an identifier column spacing out assay values
# can merge column renaming dictionaries and process in one go
new_cols.update(new_cols_2)

# There's also a 5th assay completed for the xylanases we'll deal with later:

In [8]:
# Merge all data into tidy spreadsheet

df = pd.DataFrame()
rowsum = 0
for sheet_index in range(1, len(sheet_names)):  # skip master sheet
    
    col_update = new_cols.copy()
    # Deal with Assay 5 in Xylanase
    if sheet_names[sheet_index] == 'xylanase_1':
        col_update.update({'Unnamed: 34': 'assay_5_value'})
#         print(col_update)
    
    _df = excel_raw.parse(excel_raw.sheet_names[sheet_index]).dropna(thresh=5)  # protease_1 has extra rows
    _df.rename(columns=col_update, inplace=True)  
    _df['run_label'] = sheet_names[sheet_index]
    print(sheet_names[sheet_index], len(_df))
    df = df.append(_df, sort=False)
    
df.to_csv("data/191203_combined_data.csv")

amylase_1 96
amylase_2 91
amylase_1_10x 96
amylase_2_10x 91
protease_1 96
protease_2 32
xylanase_1 96
xylanase_1_50x 96
lipase_1 96
lipase_2 96
positive_1 96
positive_2 51
positive_amy_10x 32
positive_xyl_50x 30
negative 64


### Go through and look at signal peptides

In [9]:
# Function for checking if SP exists
from Bio.pairwise2 import format_alignment
from Bio import pairwise2

all_sps = list(set(df['SP expected DNA'].values))

def check_sp(row, verbose=True, return_sp=False):
    ''' Given row in data, see if there's any SP sequence
    '''
    seq = row['Full Sequencing']

    sp_exists = False
    # SP alignment against expected
    best_sp = row['SP expected DNA']
    best_score = pairwise2.align.localxs(best_sp, seq, -1, -1, one_alignment_only=True)[0][2] 
    best_score = best_score / len(best_sp)

    # Print cases that do not match spreadsheet sent
    if best_score > 0.9:
        sp_exists = True
        if row['Correct Construct'] != True:
            aln = pairwise2.align.localxs(best_sp, seq, -1, -1, one_alignment_only=True)
            if verbose:
                print('SP found when not found previously')
                print(format_alignment(*aln[0]))

    # Look for non-expected SPs (uses all_sps) above
    if sp_exists == False:
        # Iterate through other sps
        for sp in all_sps:
            aln = pairwise2.align.localxs(sp, seq, -1, -1, one_alignment_only=True)
            score = aln[0][2] 
            norm_score = score / len(sp)
            if norm_score > 0.9:
                # If SP is found -- true
                sp_exists = True
                best_sp = sp
                # Print mistake
                if verbose:
                    print('other sp found')
                    print(format_alignment(*aln[0]))
                    print('---\n')
                break
        
    if return_sp==False:            
        return sp_exists

In [10]:
for i in range(len(df)):
    row = df.iloc[i]
    check_sp(row)

SP found when not found previously
  1 ATGGGATTCAGACTCAAAGCTCTCCTTGTCGGCTGTTTAATTTTTCTGGCGGTATCTAGCGCTATAGCCGGGGCT
    ||||||||||.|||||||||||||||||||||||||||||||||||||||||.|||||.||||||||||||||||
357 ATGGGATTCACACTCAAAGCTCTCCTTGTCGGCTGTTTAATTTTTCTGGCGGGATCTANCGCTATAGCCGGGGCT
  Score=72

SP found when not found previously
  1 ATGGTGAGTTTTAGCTCTTTACTAGCGGCAGCATCGCTAGCCGTGGTTAATGCGGGGGCT
    |||||||||||||||||||||||||||||||||||||||||.||||||||||||||||||
355 ATGGTGAGTTTTAGCTCTTTACTAGCGGCAGCATCGCTAGCTGTGGTTAATGCGGGGGCT
  Score=59

SP found when not found previously
  1 ATGAAAAAACGGTTGCACATAGGACTATTGCTATCACTCATCGCCTTCCAGGCCGGATTTGCGGGGGCT
    ||||||||||||||||||.||.||||||||||||||||||||.||||||..||||||||||||||||||
361 ATGAAAAAACGGTTGCACTTANGACTATTGCTATCACTCATCTCCTTCCNNGCCGGATTTGCGGGGGCT
  Score=64

SP found when not found previously
  1 ATGAAAAAACGGTTGCACATAGGACTATTGCTATCACTCATCGCCTTCCAGGCCGGATTTGCGGGGGCT
    |||||.|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
357 ATGAAGAAACGGTTG

In [11]:
# Apply to df
df['sp_correct'] = df.apply(lambda row: check_sp(row, verbose=False), axis = 1)
df.head()

,Glyc stock plate,Well location,Planned Protein,Planned SP,Sequence Name,Correct Construct,Full Sequencing,SP start,SP end,SP coverage,...,Average,Unnamed: 32,Assay5,assay_5_value,Unnamed: 35,Unnamed: 39,Unnamed: 40,Unnamed: 34,Average CV,sp_correct
0,Alina_Amylase1,A10,A003,NaN,>amylase1-A10-fwd_A10.ab1,True,NNNNNNNNNNTTACAATGTAGTCTTTGAAGTATTACATATGTAAGA...,357.0,434.0,99.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True
1,Alina_Amylase1,A11,A003,NaN,>amylase1-A11-fwd_A11.ab1,True,NNNNNNNNNNTTACAATGTAGTCTTTGAAGTATTACATATGTAAGA...,357.0,440.0,99.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True
2,Alina_Amylase1,A12,A003,NaN,>amylase1-A12-fwd_A12.ab1,True,NNNNNNNNNNTTACAATGTAGTCTTTGGNNGTATTACATATGTAAG...,358.0,417.0,98.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True
3,Alina_Amylase1,A1,A001,NaN,>amylase1-A1-fwd_A01.ab1,True,NNNNNNNNNNTTACANTGTAGTCTTTGAAGTATTACATATGTAAGA...,357.0,425.0,99.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True
4,Alina_Amylase1,A2,A006,NaN,>amylase1-A2-fwd_A02.ab1,True,NNNNNNNNNNNNNNNNGNANNNCNNNNNNNANNATTACATATGTAA...,359.0,427.0,99.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True


In [12]:
# Check for proteins: ### Go through and look at signal peptides

In [13]:
# Similar process for proteins

all_prots = list(set(df['Protein expected DNA'].values))

def check_prot(row, verbose=True):
    """ Given a row in protein, returns True if prot is found, otherwise False
    """
    seq = row['Full Sequencing']

    prot_exists = False
    # Prot alignment
    best_prot = row['Protein expected DNA'][:300]     # only take first few since sequencing cuts off
    best_score = pairwise2.align.localxs(best_prot, seq, -.5, -.5, one_alignment_only=True)[0][2]
    best_score = best_score / len(best_prot)

    if best_score > 0.8:
        prot_exists = True

    # Check other proteins if expected is not found
    if prot_exists == False:
        # Iterate through other prots
        for prot in all_prots:
            prot = prot[:300]
            aln = pairwise2.align.localxs(prot, seq, -.5, -.5, one_alignment_only=True)
            score = aln[0][2] 
            norm_score = score / len(prot)
            if norm_score >= 0.75:
                # If prot is found -- true
                prot_exists = True
                best_prot = prot
                # Print mistake
                if verbose:
                    print('other prot found')
                    print('original', row['Planned Protein'])
                    print('new')
                    print('best_prot')
                    print(format_alignment(*aln[0]))
                    print(norm_score)
                    print(best_prot)
                    print('---')
                    return 'further analysis'
                break
                
#     # If protein is not found, show alignment            
#     if verbose:
#         if prot_exists == False:
#             aln = pairwise2.align.localxs(best_prot, seq, -.5, -.5, one_alignment_only=True)
#             print('protein not found')
#             print(format_alignment(*aln[0]))
#             print(best_score)
#             print('---')
        
    return prot_exists

# check_prot(df.iloc[599]    # Test
# s = 'TGTCGGCTACGGAGCGTACGACTTGTATGATCTTGGCGAATTTCATCAAAAAGGTACTGTACGTACAA'

In [14]:
count = 0
for i in range(len(df)):
    row = df.iloc[i]
    a = check_prot(row)
    if a == 'further analysis':
        count += 1
        print(a)
print(str(count), ' for further analysis')

0  for further analysis


In [15]:
df['prot_correct'] = df.apply(lambda row: check_prot(row, verbose=False), axis = 1)
df.head()

,Glyc stock plate,Well location,Planned Protein,Planned SP,Sequence Name,Correct Construct,Full Sequencing,SP start,SP end,SP coverage,...,Unnamed: 32,Assay5,assay_5_value,Unnamed: 35,Unnamed: 39,Unnamed: 40,Unnamed: 34,Average CV,sp_correct,prot_correct
0,Alina_Amylase1,A10,A003,NaN,>amylase1-A10-fwd_A10.ab1,True,NNNNNNNNNNTTACAATGTAGTCTTTGAAGTATTACATATGTAAGA...,357.0,434.0,99.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True,True
1,Alina_Amylase1,A11,A003,NaN,>amylase1-A11-fwd_A11.ab1,True,NNNNNNNNNNTTACAATGTAGTCTTTGAAGTATTACATATGTAAGA...,357.0,440.0,99.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True,True
2,Alina_Amylase1,A12,A003,NaN,>amylase1-A12-fwd_A12.ab1,True,NNNNNNNNNNTTACAATGTAGTCTTTGGNNGTATTACATATGTAAG...,358.0,417.0,98.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True,True
3,Alina_Amylase1,A1,A001,NaN,>amylase1-A1-fwd_A01.ab1,True,NNNNNNNNNNTTACANTGTAGTCTTTGAAGTATTACATATGTAAGA...,357.0,425.0,99.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True,True
4,Alina_Amylase1,A2,A006,NaN,>amylase1-A2-fwd_A02.ab1,True,NNNNNNNNNNNNNNNNGNANNNCNNNNNNNANNATTACATATGTAA...,359.0,427.0,99.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,True,True


In [16]:
# Take out relevant columns
df = df[['assay_1_value', 'assay_2_value', 'assay_3_value', 'assay_4_value', 'assay_5_value', 'Correct Construct', 'Protein expected name', 'SP expected name', 'run_label','sp_correct', 'prot_correct']]

In [22]:
# Map true protein ids
enzyme_order_df['prot_seq_trunc'] = enzyme_order_df['Protein-met-sigp'].str[:80]
sp_df['prot_seq_trunc'] = sp_df['prot_seq'].str[1:81]

merge_df = enzyme_order_df.merge(sp_df, how='inner', on='prot_seq_trunc')

# The following dictionary maps the TWIST labeled enzyme ID to the corresponding SP ID
enzID_to_trueID = pd.Series(merge_df.seqID.values,index=merge_df.enzyme_id).to_dict()
assert len(list(enzID_to_trueID.keys())) == len(list(enzID_to_trueID.values())), "dics unaligned, check sequence identities"

with open("data/enzID_to_trueID.p","wb") as f:
    pickle.dump(enzID_to_trueID, f)

NameError: name 'pickle' is not defined

In [18]:
# Get nice column names
col_name_switch = dict()
for i in range(5):
    col_name_switch.update({'assay_value_' + str(i) : 'Assay' + str(i)})
df.rename(columns=col_name_switch, inplace=True)
df.rename(columns={'Protein expected name':'protein_id', 'SP expected name':'sp_id', 'Correct Construct':'correct'}, inplace=True)

df['true_prot_id'] = df['protein_id'].map(enzID_to_trueID)

In [19]:
# Update families -- but keep 10x/50x separate for now
def get_familyid(row):
    run_id = row['run_label']
    
    if run_id[-1] == 'x':
        return run_id
    else:
        return run_id.split('_')[0]   # just take family name

df['family'] = df.apply(lambda row: get_familyid(row), axis=1)

In [20]:
# Add run_id, write to csv

def get_rowid(row):
    spid = row['sp_id']
    
    if spid[:3] == 'sps':
        return row['true_prot_id'][3:] + '_' + row['sp_id'][3:]
    elif spid[:3] == 'Sig':
        return row['true_prot_id'][3:] + '_' + row['sp_id'][3:] + "_pos"
    elif spid[:3] == 'Neg':
        return row['true_prot_id'][3:] + '_' + row['sp_id'][3:] + "_neg"
    else:
        raise spid
        
df['run_id'] = df.apply(lambda row: get_rowid(row), axis=1)
df.to_csv("data/preprocessed.csv")


In [21]:
df.tail()

,assay_1_value,assay_2_value,assay_3_value,assay_4_value,assay_5_value,correct,protein_id,sp_id,run_label,sp_correct,prot_correct,true_prot_id,family,run_id
62,0.752,0.524,0.736,NaN,NaN,True,Arnold_040,Neg1,negative,True,True,seq26,negative,26_1_neg
63,0.860,0.892,0.672,NaN,NaN,True,Arnold_040,Neg1,negative,True,True,seq26,negative,26_1_neg
64,1.452,1.782,1.640,NaN,NaN,True,Arnold_040,Neg3,negative,True,True,seq26,negative,26_3_neg
65,0.574,0.644,0.622,NaN,NaN,True,Arnold_040,Neg1,negative,True,True,seq26,negative,26_1_neg
66,0.432,0.490,1.290,NaN,NaN,True,Arnold_040,Neg2,negative,True,True,seq26,negative,26_2_neg
